In [33]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from Utilities import initialize_llm

# 1. Define state (keeps track of message history)
class State(TypedDict):
    messages: Annotated[list, add_messages]

# 2. Define the tool
@tool
def add_numbers(a: int, b: int) -> int:
    """Adds two numbers together."""
    return a + b

@tool
def substract_numbers(a: int, b: int) -> int:
    """Subtracts two numbers and returns the result."""
    return a - b

tools = [add_numbers,substract_numbers]
#llm=initialize_llm("AzureOpenAI")
llm = initialize_llm("AzureOpenAI").bind_tools(tools)

In [34]:
def chatbot(state: State):
    return {"messages": [llm.invoke(state["messages"])]}

# ToolNode is a pre-built node that runs your 'tools' list
tool_node = ToolNode(tools)

In [35]:
from IPython.display import Image, display

In [45]:
from IPython.display import Image, display
import sys
import os

graph_builder = StateGraph(State)

# Add nodes to the graph
graph_builder.add_node("chatbot", chatbot)
graph_builder.add_node("tools", tool_node)

# Set the flow
graph_builder.add_edge(START, "chatbot")

# If chatbot calls a tool, go to "tools"; otherwise, END
graph_builder.add_conditional_edges("chatbot", tools_condition)

# After tools run, always return to chatbot to summarize the result
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge("chatbot", END)

# Compile the graph
graph = graph_builder.compile()
display(Image(graph.get_graph(xray=True).draw_mermaid_png(max_retries=5, retry_delay=2.0)),clear=True)
#display(Image(graph.get_graph(xray=True).draw_mermaid_png()),clear=True)
#display(Image(graph.get_graph(xray=True).draw_mermaid_png()))

ValueError: Failed to reach https://mermaid.ink/ API while trying to render your graph after 5 retries. To resolve this issue:
1. Check your internet connection and try again
2. Try with higher retry settings: `draw_mermaid_png(..., max_retries=5, retry_delay=2.0)`
3. Use the Pyppeteer rendering method which will render your graph locally in a browser: `draw_mermaid_png(..., draw_method=MermaidDrawMethod.PYPPETEER)`

In [39]:
user_input = {"messages": [("user", "What is 12 - 4?")]}
for event in graph.stream(user_input):
    print(event.values())
    for value in event.values():
        print(value["messages"][-1].content)


dict_values([{'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_VHSCJjWOTZDSeltllAo8YTdW', 'function': {'arguments': '{"a":12,"b":4}', 'name': 'substract_numbers'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 83, 'total_tokens': 103, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_f97eff32c5', 'id': 'chatcmpl-CziDEyOGgeSTabBhgA9KEWq8lgKgC', 'service_tier': None, 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': F

In [ ]:
from IPython.display import Image, display
import sys
import os
 display(Image(graph.get_graph(xray=True).draw_mermaid_png()),clear=True)

ValueError: Failed to reach https://mermaid.ink/ API while trying to render your graph after 1 retries. To resolve this issue:
1. Check your internet connection and try again
2. Try with higher retry settings: `draw_mermaid_png(..., max_retries=5, retry_delay=2.0)`
3. Use the Pyppeteer rendering method which will render your graph locally in a browser: `draw_mermaid_png(..., draw_method=MermaidDrawMethod.PYPPETEER)`